# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [42]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [43]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [44]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [45]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [46]:
# Create a connection to a Cassandra instance
from cassandra.cluster import Cluster
cluster = Cluster(["127.0.0.1"])

# Initialize a session
session = cluster.connect()

#### Create Keyspace

In [47]:
# Creating the Keyspace which we'll work on 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS song_events
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [48]:
# Setting the keyspace created above on the initialized session
try:
    session.set_keyspace("song_events")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [49]:
query1 = """SELECT artist_name, song, length FROM song_played_by_session WHERE session_id=338 AND item_in_session=4"""

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182                   

In [50]:
query2= "SELECT artist_name, song, first_name, last_name FROM song_played_by_user WHERE user_id=10 and session_id=182"

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [51]:
query3 = "SELECT first_name, last_name FROM users_who_listened_a_song WHERE song = 'All Hands Against His Own'"

### Question 1 Resolution
#### Creating the necessary table and inputing data
For table song_played_by_session, the session_id was used as a partition key because the queries will filter by this column. item_in_session was used as clustering column to help make up a unique key, since an equal session_id could appear many times in the table, and also in order to filter the query also by this column.

In [52]:
# Create Table
create_table_statement = "CREATE TABLE IF NOT EXISTS song_played_by_session "
create_table_statement += "(session_id int, item_in_session int, artist_name text, song text, length double, PRIMARY KEY(session_id, item_in_session))"
session.execute(create_table_statement)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Inserting data into the newly created table
        query = "INSERT INTO song_played_by_session (session_id, item_in_session, artist_name, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Answering the question

In [53]:
rows = session.execute(query1)
for row in rows:
    print(row.artist_name, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Question 2 Resolution
#### Creating the necessary table and inputing data
For table song_played_by_user, the columns user_id and session_id were used as a composite partition key because the queries will filter by this column and also to create a single primary key, once user_id can appear many times in the table. item_in_session was used as clustering column to sort the table by this column.

In [54]:
# Create Table
create_table_statement = "CREATE TABLE IF NOT EXISTS song_played_by_user "
create_table_statement += "(user_id int, session_id int, item_in_session int, artist_name text, song text, first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"
session.execute(create_table_statement)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Inserting data into the newly created table
        query = "INSERT INTO song_played_by_user (user_id, session_id, item_in_session, artist_name, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Answering the question

In [55]:
rows = session.execute(query2)
for row in rows:
    print(row.artist_name, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Question 3 Resolution
#### Creating the necessary table and inputing data
For table users_who_listened_a_song, the song was used as a partition key because the queries will filter by this column. user_id was used as clustering column to help make up a unique key.

In [56]:
# Create Table
create_table_statement = "CREATE TABLE IF NOT EXISTS users_who_listened_a_song "
create_table_statement += "(song text, user_id int, first_name text, last_name text, PRIMARY KEY(song, user_id))"
session.execute(create_table_statement)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Inserting data into the newly created table
        query = "INSERT INTO users_who_listened_a_song (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Answering the question

In [57]:
rows = session.execute(query3)
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [58]:
for table in ["song_played_by_session", "song_played_by_user", "users_who_listened_a_song"]:
    session.execute(f"DROP TABLE {table}")

### Close the session and cluster connection¶

In [59]:
session.shutdown()
cluster.shutdown()